In [1]:
import cv2
import numpy as np
import tensorflow as tf
import mediapipe as mp
import time

In [2]:
# Define
mp_holistic = mp.solutions.holistic 
mp_drawing = mp.solutions.drawing_utils 

def mediapipe_detection (image, model):    
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)      # Convert the image from BGR to RGB color space
    image.flags.writeable = False                       # Set the writeable flag of the image to False to prevent modification   
    results = model.process(image)                      # Process the image using the provided model (presumably the holistic model)   
    image.flags.writeable = True                        # Set the writeable flag of the image back to True for further processing   
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)      # Convert the image back from RGB to BGR color space
    return image, results

def draw_styled_landmarks(image, results):
    # Draw left hand Connection
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                mp_drawing.DrawingSpec(color = (121, 22, 76), thickness = 2, circle_radius = 4),
                                mp_drawing.DrawingSpec(color = (121, 44, 250), thickness =2, circle_radius =2)
                                ) 
    
    # Draw right hand Connection   
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                mp_drawing.DrawingSpec(color = (245, 117, 66), thickness = 1, circle_radius = 4),
                                mp_drawing.DrawingSpec(color = (245, 66, 230), thickness =1, circle_radius =2)
                                )   
    
def extract_keypoints(results):
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([lh, rh])

# Daftar class actions
actions = np.array(['A', 'B', 'C', 'D', 'E', 
                    'F', 'G', 'H', 'I', 'J', 
                    'K', 'L', 'M', 'N', 'O', 
                    'P', 'Q', 'R', 'S', 'T', 
                    'U', 'V', 'W', 'X', 'Y', 
                    'Z', 'Halo', 'Perkenalkan', 
                    'Nama', 'Saya', 'Senang', 'Bertemu', 'Kamu' ]) 

In [3]:
ls

 Volume in drive D is Data
 Volume Serial Number is 300E-DC17

 Directory of d:\capstone_project_bangkit\machine-learning

15/06/2024  11:06    <DIR>          .
15/06/2024  10:35    <DIR>          ..
15/06/2024  10:51           314.455 dataset-create-kar.ipynb
15/06/2024  11:05    <DIR>          model
13/06/2024  11:55            12.622 testing-convLSTM-realTime.ipynb
15/06/2024  10:58    <DIR>          train-code
               2 File(s)        327.077 bytes
               4 Dir(s)   4.120.920.064 bytes free


In [9]:
model = tf.keras.models.load_model("33C_ConvLSTM_acc098_loss01_s100_2.h5")

In [10]:
# Get the number of classes
output_layer = model.layers[-1]
num_classes = output_layer.units if isinstance(output_layer, tf.keras.layers.Dense) else None

print(f"Number of classes: {num_classes}")

Number of classes: 33


In [11]:
# Initialize variables
sequence = []
sentence = []
predictions = []

threshold = 0.95

state = 'countdown'
countdown_time = 3
display_time = 2
start_time = time.time()

cap = cv2.VideoCapture(0)
mp_holistic = mp.solutions.holistic

# Helper function to show countdown
def show_countdown(image, seconds_left):
    cv2.putText(image, f'Memulai prediksi dalam {seconds_left}..', (100, 250), 
                cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 4, cv2.LINE_AA)

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        # Read frame
        ret, frame = cap.read()

        current_time = time.time()
        
        if state == 'countdown':
            seconds_left = countdown_time - int(current_time - start_time)
            image = frame.copy()
            show_countdown(image, seconds_left)
            
            if seconds_left <= 0:
                state = 'collecting'
                start_time = current_time
                sequence = []
        
        elif state == 'collecting':
            # Perform detection
            image, results = mediapipe_detection(frame, holistic)
            draw_styled_landmarks(image, results)
            
            # Prediction logic
            keypoints = extract_keypoints(results)
            sequence.append(keypoints)
            
            if len(sequence) == 30:
                sequence_array = np.expand_dims(sequence, axis=-1)  # Add the last dimension
                sequence_array = np.expand_dims(sequence_array, axis=-1)  # Add one more dimension to match (None, 30, 126, 1, 1)
                res = model.predict(np.expand_dims(sequence_array, axis=0))[0]

                print(actions[np.argmax(res)])
                predictions.append(np.argmax(res))
                
                # Update sentence based on prediction
                if np.unique(predictions[-28:])[0] == np.argmax(res):
                    if res[np.argmax(res)] > threshold:
                        if len(sentence) > 0:
                            if actions[np.argmax(res)] != sentence[-1]:
                                sentence.append(actions[np.argmax(res)])
                        else:
                            sentence.append(actions[np.argmax(res)])
                
                if len(sentence) > 5:
                    sentence = sentence[-5:]

                state = 'displaying'
                start_time = current_time
        
        elif state == 'displaying':
            # Display the prediction
            if len(sentence) > 0:
                cv2.putText(image, ' '.join(sentence), (3, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            if (current_time - start_time) > display_time:
                state = 'countdown'
                start_time = current_time

        # Show the frame
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


1/1 [==============================] - 1s 594ms/step
M
1/1 [==============================] - 0s 45ms/step
N
1/1 [==============================] - 0s 49ms/step
Z


In [8]:
# Initialize variables
sequence = []
sentence = []
predictions = []

threshold = 0.95

state = 'countdown'
countdown_time = 3
display_time = 2
start_time = time.time()

cap = cv2.VideoCapture(0)
mp_holistic = mp.solutions.holistic

# Helper function to show countdown
def show_countdown(image, seconds_left):
    cv2.putText(image, f'Memulai prediksi dalam {seconds_left}..', (100, 250), 
                cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 4, cv2.LINE_AA)

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        # Read frame
        ret, frame = cap.read()

        current_time = time.time()
        
        if state == 'countdown':
            seconds_left = countdown_time - int(current_time - start_time)
            image = frame.copy()
            show_countdown(image, seconds_left)
            
            if seconds_left <= 0:
                state = 'collecting'
                start_time = current_time
                sequence = []
        
        elif state == 'collecting':
            # Perform detection
            image, results = mediapipe_detection(frame, holistic)
            draw_styled_landmarks(image, results)
            
            # Prediction logic
            keypoints = extract_keypoints(results)
            sequence.append(keypoints)
            
            if len(sequence) == 30:
                sequence_array = np.expand_dims(sequence, axis=-1)  # Add the last dimension
                sequence_array = np.expand_dims(sequence_array, axis=-1)  # Add one more dimension to match (None, 30, 126, 1, 1)
                res = model.predict(np.expand_dims(sequence_array, axis=0))[0]

                print(actions[np.argmax(res)])
                predictions.append(np.argmax(res))
                
                # Update sentence based on prediction
                if np.unique(predictions[-28:])[0] == np.argmax(res):
                    if res[np.argmax(res)] > threshold:
                        if len(sentence) > 0:
                            if actions[np.argmax(res)] != sentence[-1]:
                                sentence.append(actions[np.argmax(res)])
                        else:
                            sentence.append(actions[np.argmax(res)])
                
                if len(sentence) > 5:
                    sentence = sentence[-5:]

                state = 'displaying'
                start_time = current_time
        
        elif state == 'displaying':
            # Display the prediction
            if len(sentence) > 0:
                cv2.putText(image, ' '.join(sentence), (3, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            if (current_time - start_time) > display_time:
                state = 'countdown'
                start_time = current_time

        # Show the frame
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


1/1 [==============================] - 0s 42ms/step
Halo
1/1 [==============================] - 0s 41ms/step
Perkenalkan
1/1 [==============================] - 0s 40ms/step
Nama
1/1 [==============================] - 0s 42ms/step
Z
1/1 [==============================] - 0s 40ms/step
Z
1/1 [==============================] - 0s 46ms/step
R
1/1 [==============================] - 0s 44ms/step
Z
1/1 [==============================] - 0s 43ms/step
Z
1/1 [==============================] - 0s 44ms/step
R
1/1 [==============================] - 0s 43ms/step
Z
1/1 [==============================] - 0s 42ms/step
Saya
1/1 [==============================] - 0s 40ms/step
Z
1/1 [==============================] - 0s 45ms/step
Senang
1/1 [==============================] - 0s 45ms/step
Bertemu
1/1 [==============================] - 0s 48ms/step
Kamu
